<a href="https://colab.research.google.com/github/Khuliso877/Week-6-AI/blob/main/edge_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================
# 1. SETUP
# ============================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

print("TensorFlow version:", tf.__version__)

# ============================
# 2. LOAD DATASET
# (Using TF Flowers dataset just to simulate 2-class recyclable task)
# ============================
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

# The .tgz file extracts into a directory named 'flower_photos', which itself contains the class subdirectories.
# We need to point data_dir to the actual directory containing the classes.
data_dir = data_dir / 'flower_photos'

# The `class_names` passed did not match the names of the subdirectories.
# The flower_photos dataset contains 5 classes: daisy, dandelion, roses, sunflowers, tulips.
# To resolve the ValueError, we must use the actual class names from the dataset.
# This will make it a 5-class classification problem.
CLASS_NAMES = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])

num_classes = len(CLASS_NAMES)

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels="inferred", # Corrected to "inferred" to get labels from subdirectories
    label_mode="int",  # Use "int" for integer-encoded labels suitable for SparseCategoricalCrossentropy
    class_names=CLASS_NAMES
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels="inferred", # Corrected to "inferred" to get labels from subdirectories
    label_mode="int",  # Use "int" for integer-encoded labels
    class_names=CLASS_NAMES
)

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# ============================
# 3. BUILD A LIGHTWEIGHT CNN
# ============================
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    # Changed output layer for multi-class classification
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    # Changed loss function for multi-class classification
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()

# ============================
# 4. TRAIN MODEL
# ============================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# ============================
# 5. EVALUATE MODEL
# ============================
loss, acc = model.evaluate(val_ds)
print("Validation Accuracy:", acc)

# ============================
# 6. CONVERT TO TFLITE
# ============================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("recycle_classifier.tflite", "wb").write(tflite_model)
print("TFLite model saved!")

# ============================
# 7. RUN TFLITE INFERENCE
# ============================
interpreter = tf.lite.Interpreter(model_path="recycle_classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess a custom image
image_path = "/content/24781114_bc83aa811e_n.jpg" # Using one of the provided images
img = tf.keras.preprocessing.image.load_img(
    image_path, target_size=(img_height, img_width)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

# Ensure the input tensor has the correct data type (float32)
input_dtype = input_details[0]['dtype']
if img_array.dtype != input_dtype:
    img_array = tf.cast(img_array, input_dtype)

interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

# For multi-class classification, get the index of the class with the highest probability
predicted_class_index = np.argmax(prediction[0])
predicted_class_name = CLASS_NAMES[predicted_class_index]

print(f"Prediction probabilities for {image_path}: {prediction[0]}")
print(f"Predicted class: {predicted_class_name}")

TensorFlow version: 2.19.0
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,638,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,662,373 (6.34 MB)

 Trainable params: 1,662,373 (6.34 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.3400 - loss: 1.4847 - val_accuracy: 0.5763 - val_loss: 1.0900
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.5857 - loss: 1.0328 - val_accuracy: 0.6172 - val_loss: 1.0156
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.6743 - loss: 0.8611 - val_accuracy: 0.5858 - val_loss: 1.0776
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.7423 - loss: 0.7171 - val_accuracy: 0.5926 - val_loss: 1.1056
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.8073 - loss: 0.5251 - val_accuracy: 0.6403 - val_loss: 1.0313
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 379ms/step - accuracy: 0.6436 - loss: 1.0922
Validation Accuracy: 0.640326976776123
Saved artifact at '/tmp/tmpywr0mj81'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.flo

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
import shutil
import os

# Define new paths for simulated binary classes
recyclable_dir = data_dir / 'Recyclable'
not_recyclable_dir = data_dir / 'Not Recyclable'

# Create new directories if they don't exist
os.makedirs(recyclable_dir, exist_ok=True)
os.makedirs(not_recyclable_dir, exist_ok=True)

# Move existing images to simulate a binary dataset
# For demonstration, let's say 'roses' and 'tulips' are 'Recyclable',
# and 'daisy', 'dandelion', 'sunflowers' are 'Not Recyclable'.

# Ensure CLASS_NAMES is current after previous run before redistribution logic
current_class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])

# Move images for 'Recyclable' category
for class_name in ['roses', 'tulips']:
    if class_name in current_class_names:
        source_path = data_dir / class_name
        for img_file in source_path.iterdir():
            if img_file.is_file():
                shutil.move(str(img_file), str(recyclable_dir / img_file.name))
        shutil.rmtree(source_path) # Remove original class directory

# Move images for 'Not Recyclable' category
for class_name in ['daisy', 'dandelion', 'sunflowers']:
    if class_name in current_class_names:
        source_path = data_dir / class_name
        for img_file in source_path.iterdir():
            if img_file.is_file():
                shutil.move(str(img_file), str(not_recyclable_dir / img_file.name))
        shutil.rmtree(source_path) # Remove original class directory

print("Images redistributed for simulated binary classification.")


Images redistributed for simulated binary classification.


In [ ]:
# ============================
# 1. SETUP
# ============================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

print("TensorFlow version:", tf.__version__)

# ============================
# 2. LOAD DATASET
# (Using TF Flowers dataset just to simulate 2-class recyclable task)
# ============================
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

# The .tgz file extracts into a directory named 'flower_photos', which itself contains the class subdirectories.
# We need to point data_dir to the actual directory containing the classes.
data_dir = data_dir / 'flower_photos'

# Use two classes to simulate "recyclable vs not"
CLASS_NAMES = ["Not Recyclable", "Recyclable"] # For binary classification
num_classes = len(CLASS_NAMES)

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary", # Changed to "binary" for 2 classes
    class_names=CLASS_NAMES
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary", # Changed to "binary" for 2 classes
    class_names=CLASS_NAMES
)

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# ============================
# 3. BUILD A LIGHTWEIGHT CNN
# ============================
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    # Changed output layer for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    # Changed loss function for binary classification
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

model.summary()

# ============================
# 4. TRAIN MODEL
# ============================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# ============================
# 5. EVALUATE MODEL
# ============================
loss, acc = model.evaluate(val_ds)
print("Validation Accuracy:", acc)

# ============================
# 6. CONVERT TO TFLITE
# ============================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("recycle_classifier.tflite", "wb").write(tflite_model)
print("TFLite model saved!")

# ============================
# 7. RUN TFLITE INFERENCE
# ============================
interpreter = tf.lite.Interpreter(model_path="recycle_classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess a custom image
image_path = "/content/24781114_bc83aa811e_n.jpg" # Using one of the provided images
img = tf.keras.preprocessing.image.load_img(
    image_path, target_size=(img_height, img_width)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

# Ensure the input tensor has the correct data type (float32)
input_dtype = input_details[0]['dtype']
if img_array.dtype != input_dtype:
    img_array = tf.cast(img_array, input_dtype)

interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

# For binary classification, interpret the single output value
predicted_class_index = (prediction > 0.5).astype(int)[0][0]
predicted_class_name = CLASS_NAMES[predicted_class_index]

print(f"Prediction probability for {image_path}: {prediction[0][0]:.4f}")
print(f"Predicted class: {predicted_class_name}")


TensorFlow version: 2.19.0
Found 3669 files belonging to 2 classes.
Using 2936 files for training.
Found 3669 files belonging to 2 classes.
Using 733 files for validation.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 178, 178, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 89, 89, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,638,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,662,113 (6.34 MB)

 Trainable params: 1,662,113 (6.34 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.7430 - loss: 0.5508 - val_accuracy: 0.8295 - val_loss: 0.4191
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.8423 - loss: 0.3827 - val_accuracy: 0.8363 - val_loss: 0.3949
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.8532 - loss: 0.3507 - val_accuracy: 0.8513 - val_loss: 0.3763
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.8819 - loss: 0.3041 - val_accuracy: 0.8499 - val_loss: 0.3707
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9008 - loss: 0.2604 - val_accuracy: 0.8377 - val_loss: 0.3994
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 362ms/step - accuracy: 0.8611 - loss: 0.3780
Validation Accuracy: 0.8376534581184387
Saved artifact at '/tmp/tmptezkt4r9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
